In [1]:
import sys
sys.path.append(r'..\\')
from ioMicro import *

In [9]:
fl = r'\\192.168.0.127\Raw_data\Roy\HumanSVZMER_6_26_2023\Drift\driftNew_Conv_zscan__264--_set1.pkl'
drft,flds,fov,fl_ref = np.load(fl,allow_pickle=True)
drftT = [drf[0] for drf in drft]

In [88]:
save_folders = [r'\\192.168.0.127\Raw_data\Roy\HumanSVZMER_6_26_2023\MERFISH_Analysis_group1',
               r'\\192.168.0.127\Raw_data\Roy\HumanSVZMER_6_26_2023\MERFISH_Analysis_group2',
               r'\\192.168.0.127\Raw_data\Roy\HumanSVZMER_6_26_2023\MERFISH_Analysis_group3',
               r'\\192.168.0.127\Raw_data\Roy\HumanSVZMER_6_26_2023\MERFISH_Analysis_group4',
               r'\\192.168.0.127\Raw_data\Roy\HumanSVZMER_6_26_2023\MERFISH_Analysis_group5',
               r'\\192.168.0.127\Raw_data\Roy\HumanSVZMER_6_26_2023\MERFISH_Analysis_group6']
scoresRef = r'\\192.168.0.127\Raw_data\Roy\HumanSVZMER_6_26_2023\scoresRef.npy'
drift_folder = r'\\192.168.0.127\Raw_data\Roy\HumanSVZMER_6_26_2023\Drift'
segm_tag = 'H0_'
segm_folder = r'\\192.168.0.127\Raw_data\Roy\HumanSVZMER_6_26_2023\Segmentation'

th=-1.5
ncols=2
tag_save='finaldecs_'
force=False
save_folder_all = r'\\192.168.0.127\Raw_data\Roy\HumanSVZMER_6_26_2023\Combined'
fov='Conv_zscan__220'
set_='_set1'
plt_val=False


if type(scoresRef) is str: scoresRef = np.load(scoresRef,allow_pickle=True)
save_fl = save_folder_all+os.sep+os.sep+tag_save+fov.split('.')[0]+'--'+set_+'.npz'
if not os.path.exists(save_fl) or force:
    
    if not os.path.exists(save_folder_all): os.makedirs(save_folder_all)
    ### Load dapi
    dec.fl_dapi = segm_folder+os.sep+fov+'--'+segm_tag+set_+'--dapi_segm.npz'
    dic = np.load(dec.fl_dapi)
    im_segm = dic['segm']
    dec.shape = dic['shape']
    dec.shapesm = np.array(dec.im_segm_.shape)
    dec.im_segm_=stitch3d_new(im_segm,minsz = 200,maxsz = 1000,th_int=0.66,th_cover=0.8,th_max_subcell=0.66,nexpand = 5)

    ### Load point data ###############################
    all_gns_names = []
    XFF,genesFF = None,None
    for save_folder in save_folders:
        #save_folder = save_folders[0]
        dec = decoder_simple(save_folder,fov,set_)
        loaded = dec.load_decoded()
        if not loaded:
            print(save_fl, "Did not have a valid decoded file.")
            #return None
        dec.gns_names = np.array([gn+'_e' if gn in all_gns_names else gn for gn in dec.gns_names])
        all_gns_names = np.unique(np.concatenate([all_gns_names,dec.gns_names]))
    
        ### Threshold point data ###############################
        dec.ncols=ncols
        get_score_withRef(dec,scoresRef,plt_val=plt_val,gene=None,iSs = None)
        dec.th=th
        if plt_val:
            plot_statistics(dec)
        #print(dec.gns_ordered)
        
        keepf =  dec.scoreA>dec.th ### keep good score
        icodesf = dec.icodesN[keepf]
        XHfpr = dec.XH_pruned[keepf]
    
        ### Reorganize data  ###############################
        XHf = np.mean(XHfpr,axis=1)
        XF = XHf[:,[0,1,2,-5,-4,-3,-2,-1,-1,-1,-1]]
        #zc,xc,yc,bk-7,a-6,habs-5,hn-4,h-3
        XF[:,-1] = dec.scoreA[keepf]
        XF[:,-2] = np.where(keepf)[0]
        mnD = np.mean(np.linalg.norm((XHf[:,np.newaxis]-XHfpr)[:,:,:3],axis=-1),axis=-1)
        XF[:,-3]=mnD
        mnH = np.mean(np.abs((XHf[:,np.newaxis]-XHfpr)[:,:,-3]),axis=-1)
        XF[:,-4]=mnH
        genesf = dec.gns_names[icodesf]
    
        ### Load and apply drift correction
        def get_drft_dic(fl_all_drift):
            drft,flds,fov_,fl_ref= np.load(fl_all_drift,allow_pickle=True)
            return {'--'.join(fld.split(os.sep)[-2:]):drft_[0] for fld,drft_ in zip(flds,drft)}
        fl_all_drift = drift_folder+os.sep+'driftNew_'+fov+'--'+set_+'.pkl'
        drft_dic_all = get_drft_dic(fl_all_drift)
        fl_all_drift = save_folder+os.sep+'driftNew_'+fov+'--'+set_+'.pkl'
        drft_dic_group = get_drft_dic(fl_all_drift)
        key_segm = [key for key in drft_dic_all if segm_tag in key][0]
        key_align = np.intersect1d(list(drft_dic_all.keys()),list(drft_dic_group.keys()))[0]
        th1_to_h0 = drft_dic_all[key_segm]-drft_dic_all[key_align]
        th1_gr = drft_dic_group[key_align]
        tzxyf = th1_gr+th1_to_h0
        XF[:,:3]=XF[:,:3]-tzxyf
        if XFF is None:
            XFF,genesFF = XF,genesf
        else:
            XFF,genesFF = np.concatenate([XFF,XF]),np.concatenate([genesFF,genesf])
            
   

    def get_cell_id(dec,Xh):
        if not hasattr(dec,'im_segm_'): 
            good = np.ones(len(Xh),dtype=bool)
            cells_ = np.zeros(len(Xh),dtype=int)
            return cells_,good
        im_segm = dec.im_segm_
        dec.shapesm = dec.im_segm_.shape
        Xcms = Xh[:,:3]
        Xred = np.round((Xcms/dec.shape)*dec.shapesm).astype(int)
        good = ~np.any((Xred>=dec.shapesm)|(Xred<0),axis=-1)
        Xred = Xred[good]
        return im_segm[tuple(Xred.T)],good

    XF,genesf = XFF,genesFF  
    cell_id,good = get_cell_id(dec,XF)
    XF_ = np.concatenate([XF[good],cell_id[:,np.newaxis]],axis=-1)
    genesf_ = genesf[good]
    iset = int(dec.set_.split('_set')[-1])
    ifov = int(dec.fov.split('_')[-1].split('.')[0])
    isets = np.array([iset]*len(cell_id))[:,np.newaxis]
    ifovs = np.array([ifov]*len(cell_id))[:,np.newaxis]
    cell_id = cell_id[:,np.newaxis]
    XF_ = np.concatenate([XF[good],cell_id,ifovs,isets],axis=-1)

    get_xyfov(dec)
    XF_ = XF_[:,list(np.arange(XF_.shape[-1]))+[-1,-1]]
    XF_[:,-2:]=dec.xfov,dec.yfov
    header = ['z','x','y','abs_brightness','cor','brightness','color','mean_bightness_variation','mean_distance_variation',
              'index_from_XH_pruned','score','cell_id','ifov','iset','xfov','yfov']
    
    if not hasattr(dec,'im_segm_'): dec.im_segm_=np.zeros(np.array([30,3000,3000])//4)
    icells,vols = np.unique(dec.im_segm_,return_counts=True)
    cms = np.array(ndimage.center_of_mass(np.ones_like(dec.im_segm_),dec.im_segm_,icells))
    #icells,vols = np.unique(dec.im_segm_,return_counts=True)
    cms = np.array(ndimage.center_of_mass(np.ones_like(dec.im_segm_),dec.im_segm_,icells))
    cellinfo = cms[:,[0,0,0,1,2,0,0]]
    cellinfo[:,0]=icells
    cellinfo[:,1]=vols
    cellinfo[:,-2:]=dec.xfov,dec.yfov
    header_cells = ['cell_id','volm','zc','xc','yc','xfov','yfov']
    if False:
        np.savez_compressed(save_fl,XF=XF_.astype(np.float32),
                            genes = genesf_,cellinfo=cellinfo.astype(np.float32),header_cells=header_cells,header=header)

Loaded decoded: -0.31621670722961426
Loaded decoded: -0.6111452579498291
Loaded decoded: -0.8000900745391846
Loaded decoded: -0.6419408321380615
Loaded decoded: -0.6335418224334717
Loaded decoded: -0.43420958518981934


In [104]:
fls = glob.glob(r'C:\Scripts\NMERFISH\codebooks\codebook_Sonia2colorP*_group*_blank.csv')

gns = [[gn for gn in list(pd.read_csv(fl)['name']) if 'blank' not in gn] for fl in fls]
[[len(np.intersect1d(gn1,gn2))for gn1 in gns] for gn2 in gns]

[[188, 0, 0, 0, 0, 0],
 [0, 187, 0, 0, 0, 0],
 [0, 0, 187, 0, 0, 0],
 [0, 0, 0, 188, 0, 0],
 [0, 0, 0, 0, 189, 0],
 [0, 0, 0, 0, 0, 189]]

In [94]:
print(list(np.unique(all_gns_names)))

['AARD', 'ABCB1', 'ABI3BP', 'ACAT2', 'ACAT2_e', 'ACTC', 'ACTG1', 'ACTN1', 'ADAMTS17', 'ADAMTSL1', 'ADAMTSL3', 'ADARB2', 'ADGRG1', 'ADGRV1', 'AEBP1', 'AHCTF1', 'AHI1', 'AHI1_e', 'AHNAK2', 'AHNAK2_e', 'AIM2', 'AIM2_e', 'AKAP6', 'AKT3', 'ALDH1L1', 'ALOX5', 'AMBP', 'ANK2', 'ANK3', 'ANLN', 'ANXA2', 'ANXA2_e', 'AP2B1', 'APAF1', 'APBB1IP', 'APLP2', 'APOC1', 'APOD', 'APOD_e', 'APOE', 'APOE_e', 'APOO', 'APP', 'AQP4', 'ARL17B', 'ARPP21', 'ARPP21_e', 'ASIC2', 'ASIC4', 'ATOH8', 'ATP13A4', 'ATP13A4_e', 'ATP1B1', 'ATP1B3', 'ATP5E', 'ATP5E_e', 'ATP5G2', 'ATP6AP2', 'ATP6V1F', 'ATP6V1F_e', 'ATP7B', 'ATP8A2', 'ATPIF1', 'ATPIF1_e', 'B2M', 'BAD', 'BAX', 'BCAS1', 'BCHE', 'BCL-X', 'BCL2', 'BEX1', 'BEX1_e', 'BEX2', 'BEX2_e', 'BRAP', 'BRCA1', 'BRCA1_e', 'BRCA2', 'BRIP1', 'C1ORF132', 'C1ORF172', 'C1ORF172_e', 'C1QB', 'C1QC', 'C1QC_e', 'C1QL3', 'C4ORF26', 'C4ORF26_e', 'C9ORF142', 'C9ORF72', 'C9ORF72_e', 'CA1', 'CA10', 'CA2', 'CABLES1', 'CABLES1_e', 'CACNA1A', 'CACNA1D', 'CACNA1E', 'CACNA1H', 'CADM2', 'CADM2_e',

In [67]:
segm_tag = 'H0_'
segm_folder = r'\\192.168.0.127\Raw_data\Roy\HumanSVZMER_6_26_2023\Segmentation'


(166986, 11)

In [ ]:
cell_id,good = get_cell_id(dec,XF)

In [80]:
Xcms = XF[:,:3]-tzxyf
Xred = np.round((Xcms/dec.shape)*dec.shapesm).astype(int)
import napari
v = napari.Viewer()
v.add_labels(dec.im_segm_)
v.add_points(Xred,size=0.5)

<Points layer 'Xred' at 0x25d07460ca0>

In [ ]:
Conv_zscan__324--H0__set1--dapi_segm.npz

In [ ]:
def save_final_decoding(save_folder,fov,set_,scoresRef,th=-1.5,ncols=3,
                        tag_save = 'finaldecs_',
                        plt_val=False,apply_flat=True,
                        tags_smFISH=['Aldh','Sox11'],
                        genes_smFISH=[['Igfbpl1','Aldh1l1','Ptbp1'],['Sox11','Sox2','Dcx']],Hths=None,force=False,try_mode=True):
    """
    This loads the decoded points renormalizes them and picks the most confident points
    """
    if type(scoresRef) is str: scoresRef = np.load(scoresRef,allow_pickle=True)
    dec = decoder_simple(save_folder,fov,set_)
    save_fl = dec.save_folder+os.sep+os.sep+tag_save+dec.fov.split('.')[0]+'--'+dec.set_+'.npz'
    if not os.path.exists(save_fl) or force:
        #print(dec.fov,dec.set_)
        def main_subf(dec,save_fl,save_folder,fov,set_,scoresRef,th,ncols,plt_val,apply_flat,tags_smFISH,genes_smFISH,Hths,force):
            nsegm = load_segmentation(dec)
            loaded = dec.load_decoded()
            if not loaded:
                print(save_fl, "Did not have a valid decoded file.")
                return None
            if apply_flat:
                apply_flat_field(dec)
            apply_fine_drift(dec,plt_val=plt_val)
            
            #for i in range(3):
            #    apply_brightness_correction(dec)
            #get_scores(dec,plt_val=plt_val)
            dec.ncols=ncols
            get_score_withRef(dec,scoresRef,plt_val=plt_val,gene=None,iSs = None)
            dec.th=th
            #plot_1gene(dec,gene='Gad1',viewer = None)
            if plt_val:
                viewer = plot_multigenes(dec,genes=['Adcy1','Slc1a2','Psap'],colors=['r','g','b','m','c','y','w'],viewer = None,
                                         smin=1,smax=2.5,drift=dec.drift[0],
                                        resc = dec.shape/dec.im_segm_.shape)
                viewer.add_labels(dec.im_segm_);
            if plt_val:
                plot_statistics(dec)
            #print(dec.gns_ordered)

            keepf =  dec.scoreA>dec.th ### keep good score
            icodesf = dec.icodesN[keepf]
            XHfpr = dec.XH_pruned[keepf]
            XHf = np.mean(XHfpr,axis=1)
            if Hths is None:
                ICol = XHfpr[:,:,-2].astype(int)
                Hths = [np.percentile(XHfpr[ICol==icol][:,-3],15) for icol in np.unique(ICol)]
            
            XF = XHf[:,[0,1,2,-5,-4,-3,-2,-1,-1,-1,-1]]
            #zc,xc,yc,bk-7,a-6,habs-5,hn-4,h-3
            XF[:,-1] = dec.scoreA[keepf]
            XF[:,-2] = np.where(keepf)[0]
            mnD = np.mean(np.linalg.norm((XHf[:,np.newaxis]-XHfpr)[:,:,:3],axis=-1),axis=-1)
            XF[:,-3]=mnD
            mnH = np.mean(np.abs((XHf[:,np.newaxis]-XHfpr)[:,:,-3]),axis=-1)
            XF[:,-4]=mnH
            genesf = dec.gns_names[icodesf]
            
            ### deal with smFISH
            for tag_smFISH,gns_smFISH in zip(tags_smFISH,genes_smFISH):
                dec.get_XH_tag(tag=tag_smFISH)#dec.get_XH_tag(tag='Aldh1')
                if len(dec.Xh.shape)>1:
                    if len(dec.Xh)>0:
                        if apply_flat:
                            Xh = norm_brightness(dec,dec.Xh)
                        else:
                            Xh = dec.Xh

                        tags = [gn+'_smFISH' for gn in gns_smFISH]#['Igfbp_smFISH','Aldh1l1_smFISH','Ptbp1_smFISH']

                        for icol,tag_gn in enumerate(tags):
                            Xh_ = Xh[Xh[:,-2]==icol]
                            Xh_=Xh_[Xh_[:,-3]>Hths[icol]]
                            Xh_=Xh_[:,[0,1,2,-5,-4,-3,-2,-1,-1,-1,-1]]
                            Xh_[:,-1]=0
                            Xh_[:,-2]=-1
                            Xh_[:,-3]=0
                            Xh_[:,-4]=0
                            XF = np.concatenate([XF,Xh_])
                            genesf = np.concatenate([genesf,[tag_gn]*len(Xh_)])

            cell_id,good = get_cell_id(dec,XF)
            XF_ = np.concatenate([XF[good],cell_id[:,np.newaxis]],axis=-1)
            genesf_ = genesf[good]
            iset = int(dec.set_.split('_set')[-1])
            ifov = int(dec.fov.split('_')[-1].split('.')[0])
            isets = np.array([iset]*len(cell_id))[:,np.newaxis]
            ifovs = np.array([ifov]*len(cell_id))[:,np.newaxis]
            cell_id = cell_id[:,np.newaxis]
            XF_ = np.concatenate([XF[good],cell_id,ifovs,isets],axis=-1)

            get_xyfov(dec)
            XF_ = XF_[:,list(np.arange(XF_.shape[-1]))+[-1,-1]]
            XF_[:,-2:]=dec.xfov,dec.yfov
            header = ['z','x','y','abs_brightness','cor','brightness','color','mean_bightness_variation','mean_distance_variation',
                      'index_from_XH_pruned','score','cell_id','ifov','iset','xfov','yfov']
            
            if not hasattr(dec,'im_segm_'): dec.im_segm_=np.zeros(np.array([30,3000,3000])//4)
            icells,vols = np.unique(dec.im_segm_,return_counts=True)
            cms = np.array(ndimage.center_of_mass(np.ones_like(dec.im_segm_),dec.im_segm_,icells))
            #icells,vols = np.unique(dec.im_segm_,return_counts=True)
            cms = np.array(ndimage.center_of_mass(np.ones_like(dec.im_segm_),dec.im_segm_,icells))
            cellinfo = cms[:,[0,0,0,1,2,0,0]]
            cellinfo[:,0]=icells
            cellinfo[:,1]=vols
            cellinfo[:,-2:]=dec.xfov,dec.yfov
            header_cells = ['cell_id','volm','zc','xc','yc','xfov','yfov']

            np.savez_compressed(save_fl,XF=XF_.astype(np.float32),
                                genes = genesf_,cellinfo=cellinfo.astype(np.float32),header_cells=header_cells,header=header)
        if try_mode:
            try:
                main_subf(dec,save_fl,save_folder,fov,set_,scoresRef,th,ncols,plt_val,apply_flat,tags_smFISH,genes_smFISH,Hths,force)
            except:
                print("Failed",dec.fov,dec.set_)
        else:
            main_subf(dec,save_fl,save_folder,fov,set_,scoresRef,th,ncols,plt_val,apply_flat,tags_smFISH,genes_smFISH,Hths,force)
    

In [84]:
get_xyfov(dec)

In [7]:
flds

['\\\\192.168.0.127\\Raw_data\\Roy\\HumanSVZMER_6_26_2023\\plate1\\H1_MER_set1',
 '\\\\192.168.0.127\\Raw_data\\Roy\\HumanSVZMER_6_26_2023\\plate1\\H13_MER_set1',
 '\\\\192.168.0.127\\Raw_data\\Roy\\HumanSVZMER_6_26_2023\\plate2\\H1_MER_set1',
 '\\\\192.168.0.127\\Raw_data\\Roy\\HumanSVZMER_6_26_2023\\plate2\\H13_MER_set1',
 '\\\\192.168.0.127\\Raw_data\\Roy\\HumanSVZMER_6_26_2023\\plate3\\H1_MER_set1',
 '\\\\192.168.0.127\\Raw_data\\Roy\\HumanSVZMER_6_26_2023\\plate3\\H13_MER_set1',
 '\\\\192.168.0.127\\Raw_data\\Roy\\HumanSVZMER_6_26_2023\\H0__set1']